# Final project for Introduction to Quantum Computing (18-819F) Quantum Solver

Project Description:
- Scalable and Accurate Generation of Hybrid MPC Protocols with Quantum Integer Programming

## Preparation

### Bootstrap

In [ ]:
import numpy as np
import time
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp

# Initialize the account first.
service = QiskitRuntimeService()

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

### Available Backends

In [ ]:
service.backends()

### SILPH Reader

In [ ]:
from docplex.mp.model import Model
from docplex.mp.linear import LinearExpr

def parse_problem(fname):
    mdl = Model(fname)
    
    # Term Variable Name to Variable 
    # {name: (variable, cost)}
    term_var = {}
    # Conv Variable Name to Variable 
    # {name: (variable, cost)}
    conv_var = {}
    for line in open(fname):
        codes = line.split()
        if codes[0] == "VT":
            # term var
            if not codes[1] in term_var:
                v = mdl.binary_var(codes[1])
                term_var[codes[1]] = (v, float(codes[2]))
        elif codes[0] == "VC":
            # conversion var
            if not codes[1] in conv_var:
                v = mdl.binary_var(codes[1])
                conv_var[codes[1]] = (v, float(codes[2]))
            
        elif codes[0] == "CA":
            # assignment constraint
            if len(codes) == 2:
                # CA term_var
                (v1, _) = term_var[codes[1]]
                mdl.add_constraint(v1 >= 1)
            elif len(codes) == 3:
                # CA term_var term_var
                (v1, _) = term_var[codes[1]]
                (v2, _) = term_var[codes[2]]
                mdl.add_constraint(v1 + v2 >= 1)
            elif len(codes) == 4:
                # CA term_var term_var term_var
                (v1, _) = term_var[codes[1]]
                (v2, _) = term_var[codes[2]]
                (v3, _) = term_var[codes[3]]
                mdl.add_constraint(v1 + v2 + v3 >= 1)
            
        elif codes[0] == "CC":
            # conversion constraint
            (v1, _) = conv_var[codes[1]]
            (v2, _) = term_var[codes[2]]
            (v3, _) = term_var[codes[3]]
            mdl.add_constraint(v1 >= v2 + v3 - 1)
            
    # Create objective funciton
    exp = mdl.linear_expr()
    for (v, cost) in term_var.values():
        exp.add(v*cost)
    for (v, cost) in conv_var.values():
        exp.add(v*cost)
    mdl.minimize(exp)
    return mdl
        

### QUBO converter

In [ ]:
from qiskit_optimization.converters import InequalityToEquality
from qiskit_optimization.converters import IntegerToBinary
from qiskit_optimization.converters import LinearEqualityToPenalty

In [ ]:
ineq2eq = InequalityToEquality()
int2bin = IntegerToBinary()
lineq2penalty = LinearEqualityToPenalty()
def to_qubo(qp):
    qp_eq = ineq2eq.convert(qp)
    qp_eq_bin = int2bin.convert(qp_eq)
    qubo = lineq2penalty.convert(qp_eq_bin)
    return qubo
    

#### Convert to Ising

To Ising

In [ ]:
# qubitOp, offset = qubo.to_ising()
# print("Offset:", offset)
# print("Ising Hamiltonian:")
# print(str(qubitOp))

### Test case

In [ ]:
mdl = parse_problem("./toy_1_reduce.txt")
# mdl = parse_problem("./toy_2.txt")
# mdl = parse_problem("./biomatch_4.txt")
# mdl = parse_problem("./biomatch_16.txt")
# mdl = parse_problem("./kmeans.txt")

# print(mdl.export_as_lp_string())
qp = from_docplex_mp(mdl)
qubo = to_qubo(qp)

### Graphing

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# def plot_enumerate(results, fig_name, title=None):

#     plt.figure()

#     energies = [datum.energy for datum in results.data(
#         ['energy'], sorted_by=None)]
    
#     if results.vartype == 'Vartype.BINARY':
#         samples = [''.join(c for c in str(datum.sample.values()).strip(
#             ', ') if c.isdigit()) for datum in results.data(['sample'], sorted_by=None)]
#         plt.xlabel('bitstring for solution')
#     else:
#         samples = np.arange(len(energies))
#         plt.xlabel('solution')

#     plt.bar(samples,energies, color=(0.2, 0.4, 0.6, 0.6))
#     plt.xticks(rotation=90)
#     plt.ylabel('Energy')
#     plt.title(str(title))
#     print("minimum energy:", min(energies))
#     plt.savefig(fig_name)


# def plot_energies(results, title=None):
#     energies = results.data_vectors['energy']
#     occurrences = results.data_vectors['num_occurrences']
#     counts = Counter(energies)
#     total = sum(occurrences)
#     counts = {}
#     for index, energy in enumerate(energies):
#         if energy in counts.keys():
#             counts[energy] += occurrences[index]
#         else:
#             counts[energy] = occurrences[index]
#     for key in counts:
#         counts[key] /= total
#     df = pd.DataFrame.from_dict(counts, orient='index').sort_index()
#     df.plot(kind='bar', legend=None)

#     plt.xlabel('Energy')
#     plt.ylabel('Probabilities')
#     plt.title(str(title))
#     plt.show()
#     print("minimum energy:", min(energies))

# def parse_energy(results, name):
#     energies = results.data_vectors['energy']
#     occurrences = results.data_vectors['num_occurrences']
#     counts = Counter(energies)
#     total = sum(occurrences)
#     counts = {}
#     for index, energy in enumerate(energies):
#         if energy in counts.keys():
#             counts[energy] = [energy, counts[energy][1] + occurrences[index], name]
#         else:
#             counts[energy] = [energy, occurrences[index], name]
#     return counts

# def plot_density(result1, result2):
#     counts_1 = parse_energy(result1, "Simulated Annealing")
#     counts_2 = parse_energy(result2, "Quantum Annealing")
#     plt.figure(figsize=[8, 5])
#     df_1 = pd.DataFrame.from_dict(counts_1, orient='index').sort_index()
#     df_2 = pd.DataFrame.from_dict(counts_2, orient='index').sort_index()
#     df = pd.concat([df_1, df_2])
#     sns.set(style="whitegrid", color_codes=True)
#     ax = sns.stripplot(
#         data=df,
#         x=2, y=0, size=2, jitter=0.2
#     )
#     ax.set(xlabel='Methods', ylabel='Energy')
#     plt.savefig("./graph_quantum_annealing_1000_biomatch_4.pdf")

    
    

In [ ]:
!echo -e "\n" | dwave setup -a

In [ ]:
# print(DWaveSamples.info)

### QAOA (qiskit)

In [ ]:
from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.utils.algorithm_globals import algorithm_globals
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.problems.variable import VarType
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp

In [ ]:
algorithm_globals.random_seed = 12345
quantum_instance = QuantumInstance(
    provider.get_backend("ibmq_qasm_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
    shots = 4000
)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0.0, 1.0])
exact_mes = NumPyMinimumEigensolver()

In [ ]:
qaoa = MinimumEigenOptimizer(qaoa_mes)
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [ ]:
# exact_result = exact.solve(qubo)
# print(exact_result.prettyprint())

In [ ]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result.prettyprint())

In [ ]:
from qiskit_optimization.algorithms import WarmStartQAOAOptimizer

In [ ]:
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0.0, 1.0])
ws_qaoa = WarmStartQAOAOptimizer(
    pre_solver=CplexOptimizer(), relax_for_pre_solver=True, qaoa=qaoa_mes, epsilon=0.0
)

In [ ]:
ws_result = ws_qaoa.solve(qubo)
print(ws_result.prettyprint())

In [ ]:
# rqaoa = RecursiveMinimumEigenOptimizer(qaoa, min_num_vars=1, min_num_vars_optimizer=exact)
# rqaoa_result = rqaoa.solve(qubo)
# print(rqaoa_result.prettyprint())

In [ ]:
!pip install 'qiskit-optimization[cplex]' --quiet

In [ ]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer